In [ ]:
%cd /content/drive/MyDrive/research
!git clone https://github.com/intel-isl/MiDaS.git

/content/drive/MyDrive/research
Cloning into 'MiDaS'...
remote: Enumerating objects: 250, done.
remote: Counting objects: 100% (250/250), done.
remote: Compressing objects: 100% (173/173), done.
remote: Total 394 (delta 58), reused 202 (delta 37), pack-reused 144
Receiving objects: 100% (394/394), 231.02 KiB | 2.69 MiB/s, done.
Resolving deltas: 100% (139/139), done.


In [ ]:
%cd /content/drive/MyDrive/research/MiDaS/

# this is the test code from the repository
!python run.py

In [ ]:
%cd /content/drive/MyDrive/research/MiDaS/

# this is the custom code


import cv2
import numpy as np

from torchvision.transforms import Compose
from midas.midas_net import MidasNet
from midas.midas_net_custom import MidasNet_small
from midas.transforms import Resize, NormalizeImage, PrepareForNet

def concat():

    test_img = "/home/kie/Downloads/Telegram Desktop/IMG_0649.MOV"

    folder = "IMG_0655"
    # Create a VideoCapture object
    cap = cv2.VideoCapture("/home/kie/Downloads/output/" + folder + "/com.mp4")
    orig = cv2.VideoCapture(test_img)

    # Check if camera opened successfully
    if (not cap.isOpened() or not orig.isOpened()):
        print("Unable to read camera feed")

    # Default resolutions of the frame are obtained.The default resolutions are system dependent.
    # We convert the resolutions from float to integer.
    frame_width = 1920
    frame_height = 1080
    print("video shape", frame_width, frame_height)

    # Define the codec and create VideoWriter object.The output is stored in 'outpy.avi' file.
    out = cv2.VideoWriter('outpy.avi', cv2.VideoWriter_fourcc(
        'M', 'J', 'P', 'G'), 60, (frame_width, frame_height))
    buf = np.zeros((frame_height, frame_width, 3), dtype='uint8')

    while(True):
        ret1, frame1 = cap.read()
        ret2, frame2 = orig.read()

        if ret1 == False and ret2 == False:
            out.write(buf)
            break
        else:
            if ret1:
                frame = cv2.rotate(frame1, cv2.ROTATE_90_CLOCKWISE)
                # print(frame.shape)
                print(buf.shape)
                np.copyto(buf[:1920, :1080, :], frame)
                # print(buf)
                # break
            if ret2:
                # frame = cv2.rotate(frame2, cv2.ROTATE_90_CLOCKWISE)
                np.copyto(buf[:1920, 1080:, :], frame2)

        out.write(buf)
        # cv2.imshow('frame',buf)

    # When everything done, release the video capture and video write objects
    cap.release()
    out.release()
    orig.release()

    # Closes all the frames
    cv2.destroyAllWindows()


def initialize(model_path, optimize=True):
    """Run MonoDepthNN to compute depth maps.

    Args:
        input_path (str): path to input folder
        output_path (str): path to output folder
        model_path (str): path to saved model
    """
    print("initialize")

    # select device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("device: %s" % device)
   
    model = MidasNet(model_path, non_negative=True)
    net_w, net_h = 384, 384  
    
    transform = Compose(
        [
            Resize(
                net_w,
                net_h,
                resize_target=None,
                keep_aspect_ratio=True,
                ensure_multiple_of=32,
                resize_method="upper_bound",
                image_interpolation_method=cv2.INTER_CUBIC,
            ),
            NormalizeImage(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            PrepareForNet(),
        ]
    )

    model.eval()
    
    if optimize==True:
        rand_example = torch.rand(1, 3, net_h, net_w)
        model(rand_example)
        traced_script_module = torch.jit.trace(model, rand_example)
        model = traced_script_module
    
        if device == torch.device("cuda"):
            model = model.to(memory_format=torch.channels_last)  
            model = model.half()

    model.to(device)

    return model, transform

   

def computeDepth(model, transform):
     # get input
    img_names = glob.glob(os.path.join(input_path, "*"))
    num_images = len(img_names)

    # create output folder
    os.makedirs(output_path, exist_ok=True)

    print("start processing")

    for ind, img_name in enumerate(img_names):

        print("  processing {} ({}/{})".format(img_name, ind + 1, num_images))

        # input

        img = utils.read_image(img_name)
        img_input = transform({"image": img})["image"]

        # compute
        with torch.no_grad():
            sample = torch.from_numpy(img_input).to(device).unsqueeze(0)
            if optimize==True and device == torch.device("cuda"):
                sample = sample.to(memory_format=torch.channels_last)  
                sample = sample.half()
            prediction = model.forward(sample)
            prediction = (
                torch.nn.functional.interpolate(
                    prediction.unsqueeze(1),
                    size=img.shape[:2],
                    mode="bicubic",
                    align_corners=False,
                )
                .squeeze()
                .cpu()
                .numpy()
            )

        # output
        filename = os.path.join(
            output_path, os.path.splitext(os.path.basename(img_name))[0]
        )
        utils.write_depth(filename, prediction, bits=2)

    print("finished")
    


if (__name__ == "__main__"):
    concat()

/content/drive/MyDrive/research/MiDaS
Unable to read camera feed
video shape 1920 1080
